In [8]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


In [9]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [10]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# TODO

1. ✅CG, MGで時間をExtra-Pから取得できることを確認
2. ✅時間の単位を「秒」にする
    * 列「Inclusive total msec」に一部存在している”：”で区切られた値の単位は、msec ではなく分と秒を区切る”：”
3. Extra-P による予測との組み合わせを行う
    * なにをどう組み合わせるのかがわかっていないので、それは確かめる


In [11]:
date: str = "2022年9月2日"

target_list_process: list[int] = [256]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

target_rawDF_cg :pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_niter=cg_target_list_niter,
    list_nonzer=cg_target_list_nonzer,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath
)

target_rawDF_cg

,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,na,nonzer,niter,shift
0,100.0,0.00616,1:44.607,1.000000e+00,1.000000e+00,.TAU_application,256,1500000,21,100,200
1,100.0,35,1:44.607,1.000000e+00,1.334000e+03,CG,256,1500000,21,100,200
2,67.1,"40,154",1:10.141,1.010000e+02,1.024140e+05,CONJ_GRAD,256,1500000,21,100,200
3,31.9,642,"33,350",1.000000e+00,3.000000e+06,MAKEA,256,1500000,21,100,200
4,30.9,"15,810","32,279",1.500000e+06,1.321420e+08,SPRNVC,256,1500000,21,100,200
5,16.7,"17,463","17,463",3.454200e+04,0.000000e+00,MPI_Wait(),256,1500000,21,100,200
6,11.7,"12,284","12,284",3.454200e+04,0.000000e+00,MPI_Send(),256,1500000,21,100,200
7,10.5,"10,999","10,999",8.809450e+07,0.000000e+00,RANDLC,256,1500000,21,100,200
8,5.2,"5,469","5,469",4.404720e+07,0.000000e+00,ICNVRT,256,1500000,21,100,200
9,0.5,0.0388,559,1.000000e+00,3.003910e+00,INITIALIZE_MPI,256,1500000,21,100,200


In [12]:
list_Inclusive_column :list[str] = list(target_rawDF_cg["Inclusive"])

list_Inclusive_column

def convertPprofTime(input :str) -> float:
    """TAUで取得しpprofで集計したプロファイルの'Inclusive～', 'Exclusive～'列の時刻を文字列(string)として取得して、数値(float)として返す関数。単位は秒
    
    Args:
        input (str) : 文字列となっているpprofで集計したプロファイルの'Inclusive～'もしくは'Exclusive～'列の時刻

    Returns:
        float : inputを秒単位にした値
    """

    # ',' を削除
    input = input.replace(',' , '')
    # ':' の有無を確認
    # ':' のインデックスを探す
    index_colon :int = input.find(':')
    # '.' の有無を確認
    # '.' のインデックスを探す
    index_period :int = input.find('.')

    ret_seconds :float = 0
    minutes :int = 0
    seconds :float = 0

    if index_colon == -1:
        seconds = float(input)/1000
    else:
        minutes =  int(input[:index_colon])
        seconds =  float(input[index_colon+1:])

    # print(f"colon = {index_colon}, period = {index_period}\ninput = {input}\nminutes={minutes}, seconds={seconds}")

    ret_seconds = 60 * minutes + seconds

    return ret_seconds

def test_convertPprofTime():
    """convertPprofTime(input :str) -> float のテスト関数
    """

    test_case_00_input :str = "1:44.607"
    test_case_00_output_actually :float = 1 * 60 + 44 + 0.607
    test_case_00_output_expect :float = convertPprofTime(test_case_00_input)
    assert test_case_00_output_actually == test_case_00_output_expect, f"actually = {test_case_00_output_actually}, expect = {test_case_00_output_expect}"

    test_case_01_input :str = "33,350"
    test_case_01_output_actually :float = 33.350
    test_case_01_output_expect :float = convertPprofTime(test_case_01_input)
    assert test_case_01_output_actually == test_case_01_output_expect, f"actually = {test_case_01_output_actually}, expect = {test_case_01_output_expect}"

    test_case_02_input :str = "111:111"
    test_case_02_output_actually :float = 111 * 60 + 111
    test_case_02_output_expect :float = convertPprofTime(test_case_02_input)
    assert test_case_02_output_actually == test_case_02_output_expect, f"actually = {test_case_02_output_actually}, expect = {test_case_02_output_expect}"

    test_case_03_input :str = "0:123"
    test_case_03_output_actually :float = 123
    test_case_03_output_expect :float = convertPprofTime(test_case_03_input)
    assert test_case_03_output_actually == test_case_03_output_expect, f"actually = {test_case_03_output_actually}, expect = {test_case_03_output_expect}"

    test_case_04_input :str = "0.0269"
    test_case_04_output_actually :float = 0.0269 / 1000
    test_case_04_output_expect :float = convertPprofTime(test_case_04_input)
    assert test_case_04_output_actually == test_case_04_output_expect, f"actually = {test_case_04_output_actually}, expect = {test_case_04_output_expect}"


In [13]:
test_convertPprofTime()

In [14]:
%reset

Nothing done.


***

---

___

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


***

---

___